In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16, MobileNetV2, InceptionV3, NASNetLarge
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

In [18]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 32
EPOCHS = 100

In [19]:
early_stopping = EarlyStopping(patience=20, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=7,verbose=1, monitor="val_acc")

In [31]:
def createModelNASNetLarge(trainable=False, optimizer="adam"):
    base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(120, activation="softmax"))
    
    if not trainable:
        for layer in base_model.layers:
            layer.trainable = False

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
  
    return model

In [32]:
model = createModelNASNetLarge()

# Using the ImageDataGenerator

In [33]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.1],
                                   brightness_range=[0.8, 1.2],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [34]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(331, 331),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(331, 331),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(331, 331),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = test_generator, 
    validation_steps = test_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

## Evaluation

In [29]:
loss, acc = model.evaluate_generator(validation_generator,verbose=0, steps=validation_generator.samples // BATCH_SIZE)

In [ ]:
print(loss,acc)

In [31]:
model.save("nasnetlarge-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))